### Ex 1: 
Si consideri l'interprete big step delle espressioni aritmetiche visto a lezione.

Si estenda la definizione dell'AST (modificando il codice qui sotto) in modo che possa descrivere identificatori `Ide` e un costrutto `sumall (x;n1;n2) e` tale che la sintassi del linguaggio diventi:

```
Exp ::= n | Ide | Exp op Exp | (Exp) | sumall ( Ide ; n ; n ) Exp
Ide ::= x | y | z | ...
op ::= + | - | * | / 
```

**NOTA:** Per gli identificatori, definire il tipo `ide` come alias del tipo `string`

#### Soluzione:

L'espressione `sumall (x; 1; 10) 5*x` deve essere intesa come "Sommatoria per x che va da 1 a 10 di 5*x", ovvero:
```
5*1 + 5*2 + 5*3 + ... + 5*9 + 5*10
```
Vale la pena notare che l'espressione `sumall (x;5;5) x-1` non richiede di calcolare alcuna somma, poichè l'espressione `x-1` deve essere valutata una sola volta, quando x è uguale a 1. Quindi l'espressione `sumall (x;5;5) x-1`, nel nostro linguaggio delle espressioni, è concettualmente simile a `let x = 5 in x-1` in Ocaml.

Come richiede la consegna, dobbiamo estendere la sintassi astratta con due nuovi costruttori, ovvero due nuovi nodi possibili con cui poter costruire un AST:
- `Ide`, una foglia dell'AST, che non contiene nessuna altra sotto espressione. Un nodo Ide dentro l'AST indica un identificatore (o variabile che dir si voglia), e contiene un valore di tipo string, cioè il nome della variabile. Aver aggiunto il nodo Ide permette di formalizzare in un AST espressioni come `2*x` o `i+j/2`, rispettivamente come `Op(Mul, Val(2), Ide("x"))` e `Op(Div, Op(Add, Ide("i"), Ide("j")), Val(2))`. (Queste espressioni sono sintatticamente corrette, ma non hanno alcun significato, perchè manca un **quantificatore** che associ un valore a ogni variabile libera.)
- `Sumall`, un nodo dell'AST che contiene una singola sottoespressione `e`, e altre 3 informazioni aggiuntive: il nome della variabile legata, e i due estremi dell'intervallo di valori che la variabile legata dovrà assumere. Scegliamo di rappresentare i due estremi come interi, per semplicità, ma sarebbe ragionevole anche rappresentarli come sottoespressioni di tipo Val. Aver aggiunto il nodo Sumall permette di formalizzare in un AST espressioni come 
    
    `sumall (k, 1, 5) k-1` in `Sumall("x", 1, 5, Op(Sub, Ide("k"),Val(1)))`.

In [1]:
type op = Add | Sub | Mul | Div ;;
type ide = string;;
type exp = 
    | Val of int
    | Op of op*exp*exp
    | Ide of ide
    | Sumall of ide*int*int*exp;;
    (*Se definissimo sumall come ide*int*int*op, non potremmo rappresentare 
      una sommatoria come "sumall (n; 1; 100) n". Scegliamo quindi exp come 
      sottoespressione di sumall perchè è la più generale possibile.*)
    (*Potremmo in teoria definire sumall come ide*exp*exp*exp, consentendo anche
      espressioni come "sumall (x; 1+1; 5*10) x*x". Questa sintassi astratta sarebbe
      però in contrasto con la sintassi data nella consegna.*)

type op = Add | Sub | Mul | Div


type ide = string


type exp =
    Val of int
  | Op of op * exp * exp
  | Ide of ide
  | Sumall of ide * int * int * exp


In [3]:
(*Secondo questa sintassi astratta, l'espressione
  sommatoria per k che va da 1 a 5 di k-1
  (aka sumall (k, 1, 5) k-1)
  può essere formalizzata con il seguente AST:*)
Sumall ("x", 1, 5, Op(Sub, Ide("k"),Val(1)))

- : exp = Sumall ("x", 1, 5, Op (Sub, Ide "k", Val 1))


#### Ex 2:
Nel linguaggio esteso come da esercizio precedente, si implementi una funzione `subst (e:exp) (x:ide) (v:exp)` dove `e` è un'espressione, `x` è un identificatore e `v` è un'espressione che deve descrivere un valore (`Val n`). La funzione calcola l'espressione $e\{x:=v\}$ in cui tutte le occorrenze "libere" di `x` in `e` sono sostituite con `v`. 

**ATTENZIONE:** Il costrutto `sumall (x;n1;n2) e` lega le occorrenza di `x` in `e`. La sostituzione deve operare solo sulle occorrenze libere della variabile. 

val subst : exp -> ide -> exp -> exp = <fun>


**TEST:** applicare la funzione `subst` in modo da calcolare $((x+y)+(\mbox{sumall }x;1;10) \, x) \{x:=5\}$ che deve restituire $((5+y)+(\mbox{sumall }x;1;10)\, x)$.

In [10]:
let e1 = Op (Add, (Op (Add, Ide "x", Ide "y")), (Sumall ("x",1,10,Ide "x"))) ;;
subst e1 "x" (Val 5);;

val e1 : exp =
  Op (Add, Op (Add, Ide "x", Ide "y"), Sumall ("x", 1, 10, Ide "x"))


- : exp = Op (Add, Op (Add, Val 5, Ide "y"), Sumall ("x", 1, 10, Ide "x"))


#### Ex 3:

Si modifichi il codice (riportato qui sotto) dell'interprete big step delle espressioni in modo tenere conto della semantica estesa con il costrutto `sumall`, definita come segue. Per l'operazione di sostituzione $e \{x := v\}$ utilizzare la funzione `subst` definita nell'esercizio precedente.

$$
n \rightarrow_{bs} n
\qquad
\frac{E_1 \rightarrow_{bs} n_1 \quad E_2 \rightarrow_{bs} n_2 \quad n_1 \mbox{op } n_2 = n}{E_1 \, op \, E_1 \rightarrow_{bs} n}
$$

$$
\frac{e\{x:=n\} \rightarrow_{bs} n'}
{\mbox{sumall } ( x ; n ; n ) \, e \rightarrow_{bs} n'}
$$

$$
\frac{n_1<n_2 \quad e\{x:=n_1\} \rightarrow_{bs} n' \quad \mbox{sumall } ( x ; n_1+1 ; n_2 ) \, e \rightarrow_{bs} n'' \quad n = n' + n''}
{\mbox{sumall } ( x ; n_1 ; n_2 ) \, e \rightarrow_{bs} n}
$$

**NOTA1:** non sono previste regole per gli identificatori (sono gestiti dalla sostituzione).

**NOTA2:** la semantica non prevede transizioni in caso di identificatori liberi (non legati da `sumall`) o indici numerici usati nel costrutto `sumall` non sono ben definiti

In [26]:
(* interprete big-step *)
let rec eval e =
    match e with
    | Val n -> Val n
    | Op (op,e1,e2) -> 
        (* chiamate ricorsive che calcolano le derivazioni per e1 ed e2 *)
        match (eval e1, eval e2) with
        | (Val n1, Val n2) -> (match op with   (* calcola n1 op n2 *)
                               | Add -> Val (n1+n2)
                               | Sub -> Val (n1-n2)
                               | Mul -> Val (n1*n2)
                               | Div -> Val (n1/n2)
                               )
        (* caso (inutile) aggiunto solo per rendere esaustivo il pattern matching *)
        | _ -> failwith "Errore impossibile che si verifichi" ;;

val eval : exp -> exp = <fun>


**TEST:** calcolare il risultato delle seguenti espressioni:

* sumall (x; 1; 10) x
* 100 + sumall (x;1;10) (sumall (y;1;10) x*y)

Risultati attesi: 55 e 3125

In [28]:
let e2 = Sumall ("x",1,10,Ide "x");;
let e3 = Op (Add, Val 100, 
                 (Sumall ("x",1,10,(Sumall ("y",1,10, Op ( Mul, Ide "x", Ide "y")))))) ;;
                 
eval e2;;
eval e3;;

val e2 : exp = Sumall ("x", 1, 10, Ide "x")


val e3 : exp =
  Op (Add, Val 100,
   Sumall ("x", 1, 10, Sumall ("y", 1, 10, Op (Mul, Ide "x", Ide "y"))))


- : exp = Val 55


- : exp = Val 3125


#### Esercizio supplementare:

Scrivere una funzione `check` che analizzi staticamente la correttezza di una espressione nel linguaggio esteso. La funzione prende l'espressione e restituisce `true` se essa non contiene variabili libere e tutti i `sumall` hanno indici che garantiscono la terminazione del calcolo, `false` altrimenti.

val check : exp -> bool = <fun>


**TEST:** applicare la funzione `check` alle seguenti espressioni:

* $((5+y)+(\mbox{sumall }x;1;10) \, x)$
* $(5+(\mbox{sumall }x;1;10) \, x)$
* $(5+(\mbox{sumall }x;10;1) \, x)$

Risultati attesi: `false`,`true`,`false`

In [32]:
let e4 = Op (Add, (Op (Add, Val 5, Ide "y")), (Sumall ("x",1,10,Ide "x"))) ;;
let e5 = Op (Add, Val 5, (Sumall ("x",1,10,Ide "x"))) ;;
let e6 = Op (Add, Val 5, (Sumall ("x",10,1,Ide "x"))) ;;

check e4;;
check e5;;
check e6;;

val e4 : exp =
  Op (Add, Op (Add, Val 5, Ide "y"), Sumall ("x", 1, 10, Ide "x"))


val e5 : exp = Op (Add, Val 5, Sumall ("x", 1, 10, Ide "x"))


val e6 : exp = Op (Add, Val 5, Sumall ("x", 10, 1, Ide "x"))


- : bool = false


- : bool = true


- : bool = false
